In [1]:
import os, sys, random, time, pickle
src_dir_ = '/home/tan/Documents/GitHub/pdpt_2022/src'
sys.path.insert(1, src_dir_)

import numpy as np
from pathlib import Path

from util import read_pickle, group_cycle_truck, manual_stop
from pdpt_route_schedule import gurobi_master_cycle, greedy_fix_MP, MP_to_SP, cpo_sub, greedy_heuristic, time_checker_cluster, calculate_SP_cost, postprocess_pdpt_solution




In [2]:
def find_key(list_dict, value):
    return  [k for item_ in list_dict for k, v in item_.items() if value in v]

In [3]:
def select_subroutes(ins, cargo_route_file, truck_pairs_to_try, undelivered_cargos, seed=0, verbose=0):


    # load data from ins
    truck = ins['truck']
    cargo = ins['cargo']
    # edges = ins['edge']
    # nodes = ins['nodes']
    # constant = ins['constant']
    # node_cargo_size_change = ins['node_cargo_size_change']
    edge_shortest = ins['edge_shortest']
    # path_shortest = ins['path_shortest']
    # single_truck_deviation = ins['single_truck_deviation']
    selected_truck = {}
    selected_node = []
    selected_edge = {}
    selected_cargo = {}

    cargo_to_truck_assignment = [{key: list(set([v[0] for v in value]))} for key, value in cargo_route_file.items()]

    # print(cargo_to_truck_assignment)
    
    # for truck_key in truck_keys_shuffle[:2]:
    for truck_key in truck_pairs_to_try:
        if verbose >0:
            print(f'========= START [PDOTW with truck {truck_key}] ========= ')

        truck_value = truck[truck_key]

        selected_truck[truck_key] = truck[truck_key]

        cargo_keys = find_key(cargo_to_truck_assignment, truck_key)
        if len(cargo_keys) > 0:
            for c_key in cargo_keys: 
                selected_cargo[c_key] = cargo[c_key]

        if truck_value[0] not in selected_node:
            selected_node.append(truck_value[0])
        if truck_value[1] not in selected_node:
            selected_node.append(truck_value[1])
    # add undelivered cargos
    for cargo_key, cargo_route in cargo_route_file.items():
        if len(cargo_route) == 0:
            selected_cargo[cargo_key] = cargo[cargo_key]
            
    if len(undelivered_cargos) > 0:
        # print('+++ add undelivered cargos to the sub-problem')
        for cargo_key, cargo_value in undelivered_cargos.items():
            if cargo_key not in selected_cargo.keys():
                selected_cargo[cargo_key] = cargo_value

    for v in selected_cargo.values():
        if v[3] not in selected_node:
            selected_node.append(v[3])
        if v[4] not in selected_node:
            selected_node.append(v[4])

    edges_ = list(set([(i,j) for i in selected_node for j in selected_node]))

    for i,j in edges_:
        selected_edge[(i,j)] = edge_shortest[(i,j)]


    return (selected_cargo, selected_truck, selected_node, selected_edge)

In [4]:
def pdpt_route_schedule_decomposition(path_, ins, subroutes, verbose):
    """
    The main function of PDPT decomposition
    input: ct = count
    
    output:
        infeasible_clusters
        removed_cargo_MP
        removed_cargo_SP

    Three steps to handle removed cargos:
        1. add a removed cargo to a different cluster, resolve the MP-SP
        2. add all removed cargos to an unused truck
        3. LNS
    """    



    truck = ins['truck']
    cargo = ins['cargo']
    constant = ins['constant']
    # single_truck_deviation = ins['single_truck_deviation']
        
    selected_cargo, selected_truck, selected_node, selected_edge = subroutes


    # Setting parameters for clustering cargo and truck 
    runtime = 100
    # count_node_cluster = 10
    # conflict_obj_coefficient = 10000
    
      
    ###### Solve the MP+SP for each cluster ######
    
    time_start = time.time()
    
    removed_cargo = {}
    # solution_cluster = {}   


    ###### Initialize data structures for the solution of the current cluster ######

    # solution_created_truck = {}

    ### group cycle and non-cycle trucks
    created_truck_yCycle, created_truck_nCycle, created_truck_all = \
    group_cycle_truck(selected_truck) 


    ###### Master Problem Solved by Gurobi MIP ######

    # no variable ordering
    # heuristic = 0.2

    #x_sol: x^k_{ij}, if truck k visit edge (i,j) or not
    #s_sol: s^k, if truck k i used or not
    #z_sol: z^{kr}_{ij}, if truck k carries parcel r visit edge (i, j) or not
    #y_sol: y^k_r, if parcel r is carried by truck k
    #u_sol: y^r_i, if parcel r is transfered at node i
    #D_sol: D^k_i, depature time of truck k at node i

    gurobi_log_file = path_ + f'_gurobi/rasd_MP.log'
    

    obj_val_MP, runtime_MP, \
    x_sol, s_sol, z_sol, y_sol, u_sol, D_sol, Db_sol, \
    cost_truck_value, cost_travel_value, cost_transfer_value = \
    gurobi_master_cycle(constant, selected_cargo, 
        created_truck_yCycle, created_truck_nCycle, created_truck_all, 
        selected_edge, selected_node, runtime*1, gurobi_log_file, verbose)

    if obj_val_MP < 0:
        # print(f'MP infeasible, stop solving, skip to the next iter of RASD')
        # return False, {}
        selected_cargo, created_truck_all, selected_edge, selected_node, \
        selected_removed_cargo, obj_val_MP, runtime_MP, \
        x_sol, s_sol, z_sol, y_sol, u_sol, D_sol = \
        greedy_fix_MP(constant, selected_cargo, 
                    created_truck_yCycle, created_truck_nCycle, created_truck_all, 
                    selected_edge, selected_node, runtime, gurobi_log_file, verbose)
        # find removed cargo from MP
        for c in selected_removed_cargo:
            removed_cargo[c] = cargo[c]

    else:
        print(f'MP feasible, move to SP')

    
    ###### The MP should be feasible here ######

    ###### Subproblem Solved by CPO CP ######

    # convert MP solution to CP parameters
    print(f'Convert MP solution to SP parameters')

    truck_MP, truck_nodes, truck_nodes_index, cargo_in, cargo_out, \
    transfer_nodes, cargo_unload, cargo_load = \
    MP_to_SP(constant, selected_cargo, 
        created_truck_yCycle, created_truck_nCycle, created_truck_all, 
        selected_edge, selected_node,
        x_sol, s_sol, z_sol, y_sol, u_sol, D_sol)  
    

    # Evaluating the feasibility of SP
    feasibility_SP, g_sol, h_sol, D_sol = \
    cpo_sub(constant, selected_cargo, 
        created_truck_yCycle, created_truck_nCycle, created_truck_all, 
        selected_edge, selected_node,
        truck_MP, truck_nodes, truck_nodes_index, 
        cargo_in, cargo_out,
        transfer_nodes, cargo_unload, cargo_load,
        runtime)

    if feasibility_SP != 'Feasible':
        print(f'SP infeasible, repair with greedy heuristic')

        # Greedy heuristic for cargo removal
        selected_removed_cargo, \
        truck_MP, truck_nodes, truck_nodes_index, \
        cargo_in, cargo_out, \
        transfer_nodes, cargo_unload, cargo_load = \
        greedy_heuristic(constant, selected_cargo, 
            created_truck_yCycle, created_truck_nCycle, created_truck_all, 
            selected_edge, selected_node,
            truck_MP, truck_nodes, truck_nodes_index,
            cargo_in, cargo_out,
            transfer_nodes, cargo_unload, cargo_load)

        # find removed cargo from SP
        for c in selected_removed_cargo:
            removed_cargo[c] = cargo[c]
    else:
        print(f'SP Feasible, Check time constraint')
        if time_checker_cluster(constant, selected_cargo, created_truck_all, 
                            selected_edge, truck_MP, truck_nodes, 
                            cargo_unload, cargo_load, g_sol, h_sol, D_sol):
            print('Passing time constraint check!')
        else:
            print('Failed time constraint check, exit!!!!')
            print(f'Failed time constraint check, skip to the next iter of RASD')
            return False, {}
            # sys.exit()
        
    # Calculate SP costs: truck cost + traveling cost + transfer cost
    if verbose > 0:
        print(f'Compute cost after solving SP')

    truck_cost, travel_cost, transfer_cost = \
    calculate_SP_cost(constant, selected_cargo, selected_edge, 
        truck_MP, truck_nodes, 
        cargo_in, cargo_out, transfer_nodes)


    truck_used, cargo_delivered, cargo_undelivered, \
    trucks_per_cargo, cargo_in_truck, truck_route, cargo_route =  postprocess_pdpt_solution(subroutes, x_sol, s_sol, y_sol, z_sol, u_sol, verbose = 0)

    print(f'===== summary of post-processing [{selected_truck.keys()}] route+schedule solution')
    print(f'+++ cargo to truck assignment')
    for key, value in cargo_in_truck.items():
        print(f'      {key}: {value}')
    print(f'+++ undelivered cargos:\n     {removed_cargo}')

            
    if verbose >0:
        print(f'+++ truck to cargo assignment')
        for key, value in trucks_per_cargo.items():
            print(f'      {key}: {value}')
        print(f'+++ truck route')
        for key, value in truck_route.items():
            print(f'      {key}: {value}')
        print(f'+++ cargo route')
        for key, value in cargo_route.items():
            print(f'      {key}: {value}')

    res = {'MP': {'x_sol': x_sol,
                  's_sol': s_sol,
                  'z_sol': z_sol,
                  'y_sol': y_sol,
                  'u_sol': u_sol,
                  'D_sol': D_sol,
                  'Db_sol': Db_sol,
                 },
            'SP':{'g_sol': g_sol,
                  'h_sol': h_sol,
                  'D_sol': D_sol,
                 },
            'route':{'removed_cargo':removed_cargo,
                     'used_truck':truck_used,
                     'truck_route': truck_route,
                     'cargo_route': cargo_route,
                     'transfer_nodes': transfer_nodes,
                    },
            'cost':{'truck_cost': truck_cost,
                    'travel_cost': travel_cost,
                    'transfer_cost': transfer_cost,
                    }
            }

    return True, res

In [5]:
def update_pdpt_sol(pdpt_ins, pdpt_sol, res):
    ''' 
        PDPT_suprob = {'MP': {'x_sol': x_sol,
                  's_sol': s_sol,
                  'z_sol': z_sol,
                  'y_sol': y_sol,
                  'u_sol': u_sol,
                  'D_sol': D_sol,
                  'Db_sol': Db_sol,
                 },
            'SP':{'g_sol': g_sol,
                  'h_sol': h_sol,
                  'D_sol': D_sol,
                 },
            'route':{'truck_route': truck_route,
                     'cargo_route': cargo_route,
                    },
            'cost':{'truck_cost': truck_cost,
                    'travel_cost': travel_cost,
                    'transfer_cost': transfer_cost,
                    }
            }

        PDOTW = {'MIP': {'x_sol': x_sol_total,
                   'y_sol': y_sol_total,
                   'S_sol': S_sol_total,
                   'D_sol': D_sol_total,
                   'A_sol': A_sol_total,
                   'Sb_sol': Sb_sol_total,
                   'Db_sol': Db_sol_total,
                   'Ab_sol': Ab_sol_total,
                   'runtime': runtime_pdotw,
                  },
           'route': {'truck_yCycle':list(created_truck_yCycle_total.keys()),
                     'used_truck': truck_used_total,
                     'truck_route': truck_route,
                     'cargo_route': cargo_route,
                    },
           'cost': {'truck_cost' : truck_cost,
                    'travel_cost' : travel_cost,
                    },
          }
    '''

    curr_pdpt_sol = pdpt_sol.copy()
    curr_sub_pdpt_sol = res.copy()
    edge_shortest = pdpt_ins['edge_shortest']
    constant = pdpt_ins['constant']
    cargo_list= pdpt_ins['cargo']

    curr_pdpt_sol['removed_cargo'] = curr_sub_pdpt_sol['route']['removed_cargo']

    for truck_key, route in curr_sub_pdpt_sol['route']['truck_route'].items():
        curr_pdpt_sol['truck_route'][truck_key] = route
    
    for truck_key, route in curr_sub_pdpt_sol['route']['cargo_route'].items():
        curr_pdpt_sol['cargo_route'][truck_key] = route

    travel_cost = 0
    truck_cost = 0
    for truck_key, route in curr_pdpt_sol['truck_route'].items():
        for r_idx in range(len(route)-1):
            travel_cost += edge_shortest[(route[r_idx], route[r_idx+1])] * constant['truck_running_cost']
        if len(route) > 1:
            truck_cost += constant['truck_fixed_cost']

    transfer_cost = 0
    for pair in curr_sub_pdpt_sol['route']['transfer_nodes']:
        curr_pdpt_sol['transfer_nodes'].append(pair)
        node_key, cargo_key = pair
        transfer_cost += int(cargo_list[cargo_key][0] * constant['cargo_reloading_cost'])*2

    return curr_pdpt_sol, (travel_cost, truck_cost, transfer_cost)

    
        


In [6]:
# def rasd(dir_, max_success_iteration, max_iter, runtime_limit, case_num, trucks_per_subprob, seed=0):

dir_ = '/home/tan/Documents/GitHub/pdpt_2022/'
seed = 0
runtime_limit = 20*60

max_iter = 10
max_success_iteration = 10
trucks_per_subprob = 2

for case_num in range(1,6,1):
    print(f'_________case {case_num}__________')
    Path(os.path.join(dir_,  'out', 'impSol', f'case{case_num}_gurobi')).mkdir(parents=True, exist_ok=True)
    
    start_time = time.time()

    pdpt_ins_filename = os.path.join(dir_, f'data/case{case_num}.pkl')
    pdotw_sol_res_filename = os.path.join(dir_, 'out', 'iniSol', f'case{case_num}_iniSol.pkl')

    pdpt_ins = read_pickle(pdpt_ins_filename)
    truck_list = pdpt_ins['truck']
    cargo_list = pdpt_ins['cargo']

    pdotw_sol = read_pickle(pdotw_sol_res_filename)
    best_cost = sum([c_ for c_ in pdotw_sol['cost'].values()])
    cargo_route = pdotw_sol['route']['cargo_route']

    pdpt_sol = {}
    pdpt_sol['truck_route'] = pdotw_sol['route']['truck_route'].copy()
    pdpt_sol['cargo_route'] = pdotw_sol['route']['cargo_route'].copy()
    pdpt_sol['transfer_nodes'] = []
    pdpt_sol['travel_cost'] = pdotw_sol['cost']['travel_cost']
    pdpt_sol['truck_cost'] = pdotw_sol['cost']['truck_cost']
    pdpt_sol['transfer_cost'] = 0
    pdpt_sol['removed_cargo'] = 0

    cargos_in_truck = {}
    for truck_key in pdotw_sol['route']['used_truck']:
        cargos_in_truck[truck_key] = [ cargo_key  for cargo_key, c_route_ in cargo_route.items() if truck_key in [r_[0] for r_ in c_route_] ]

    truck_key_list = np.array([[truck_key, len(values)] for truck_key, values in cargos_in_truck.items() if len(values)>0], dtype=object)
    truck_key_list = truck_key_list[truck_key_list[:, 1].argsort()]  # sort by day



    used_truck_list = {}
    for truck_key in truck_key_list[:,0]:
        used_truck_list[truck_key] = truck_list[truck_key]

    undelivered_cargo_key = [key for key, value in cargo_route.items() if len(value)==0]
    undelivered_cargos = {}
    for cargo_key in undelivered_cargo_key:
        undelivered_cargos[cargo_key] = cargo_list[cargo_key]



    if len(undelivered_cargos.keys())<=0:
        least_truck = truck_key_list[0,0]
        for cargo_key in cargos_in_truck[least_truck]:
            undelivered_cargos[cargo_key] = cargo_list[cargo_key]
            del pdpt_sol['cargo_route'][cargo_key]

    del pdpt_sol['truck_route'][truck_key_list[0,0]]
    del used_truck_list[truck_key_list[0,0]]
    truck_key_list = truck_key_list[1:, :]
    truck_to_try = truck_key_list[:,0].tolist()

    selected_truck_pairs = []
    num_success_rasd_iter = 0
    iter = 0
    while num_success_rasd_iter < max_success_iteration and iter <=max_iter:
        if iter == 0:
            t1, t2 = truck_to_try[:trucks_per_subprob]
        selected_truck_pairs.append([t1,t2])
        subroutes = select_subroutes(pdpt_ins, pdotw_sol['route']['cargo_route'], [t1,t2], undelivered_cargos, seed=seed, verbose=0)
        print(f'\n++++++ instantitate PDPT subproblem on truck {list(subroutes[1].keys())}')
        feasibility_flag, sub_pdpt_sol = pdpt_route_schedule_decomposition(os.path.join(dir_, 'out', 'impSol'+f'/case{case_num}'), 
                                                                        pdpt_ins, subroutes, verbose = 0)

        feasibility_flag = False
        if feasibility_flag == False:
            # random.seed(seed+iter)
            t2 = t1
            while t2 == t1 or [t1,t2] in selected_truck_pairs:
                idx = random.randint(0, len(truck_to_try)-1)
                t2 = truck_to_try[idx]
            # truck_to_try = truck_to_try[1:]
            # t1, t2 = truck_to_try[:trucks_per_subprob]
        elif feasibility_flag == True:
            curr_sol, curr_cost = update_pdpt_sol(pdpt_ins, pdpt_sol, sub_pdpt_sol)
            if sum(curr_cost) < best_cost:
                print('+++ find better_solution in RASD')
                pdpt_sol = curr_sol.copy()
                best_cost = sum(curr_cost)
                undelivered_cargos = sub_pdpt_sol['route']['removed_cargo'].copy()
                num_success_rasd_iter += 1
                truck_to_try = [t_ for t_ in truck_to_try if t_ not in [t1,t2]]
                t1, t2 = truck_to_try[:trucks_per_subprob]
            else:
                print('+++ no better solution found, move on')
        iter += 1
        selected_truck_pairs.append(list(subroutes[1].keys()))
        if time.time() - start_time > runtime_limit:
            print(f'+++ Reaching timelimit of {runtime_limit}')
            break
        if len(truck_to_try)<2:
            print('+++ There are not enough left trucks to try RASD')
            break

    Path(os.path.join(dir_,  'out', 'impSol')).mkdir(parents=True, exist_ok=True)

    filename = os.path.join(dir_,'out','impSol')+f'/case{case_num}_impSol.pkl'
    with open(filename, 'wb') as f:
        pickle.dump(pdpt_sol, f)

_________case 1__________

++++++ instantitate PDPT subproblem on truck ['T7', 'T9']
Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-15
Set parameter TimeLimit to value 100
Set parameter LogFile to value "/home/tan/Documents/GitHub/pdpt_2022/out/impSol/case1_gurobi/rasd_MP.log"
Set parameter Heuristics to value 0.2
********************** MP Infeasible *********************
Evaluating removing ['C41'] now..................
Remove node N24
Set parameter TimeLimit to value 10
Set parameter LogFile to value "/home/tan/Documents/GitHub/pdpt_2022/out/impSol/case1_gurobi/rasd_MP.log"
Set parameter Heuristics to value 0.2
********************** MP Infeasible *********************
Evaluating removing ['C73'] now..................
Remove node N6
Set parameter TimeLimit to value 10
Set parameter LogFile to value "/home/tan/Documents/GitHub/pdpt_2022/out/impSol/case1_gurobi/rasd_MP.log"
Set parameter Heuristics to value 0.2
********************** MP Infeasib

In [7]:
# dir_ = '/home/tan/Documents/GitHub/pdpt_2022/'
# seed = 0
# runtime_limit = 20*60

# max_iter = 10
# max_success_iteration = 10
# trucks_per_subprob = 2

# for case_num in range(1,6,1):
#     print(f'_________case {case_num}__________')
#     Path(os.path.join(dir_,  'out', 'impSol', f'case{case_num}_gurobi')).mkdir(parents=True, exist_ok=True)

#     rasd(dir_, max_success_iteration, max_iter, runtime_limit, case_num, trucks_per_subprob)